### Creating SparkSession

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Recommendation_System').getOrCreate()

###Importing the dataset

In [4]:
df = spark.read.csv('/FileStore/tables/movielens_ratings.csv', inferSchema=True, header=True)

In [5]:
df.show()

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 2| 3.0| 0|
 3| 1.0| 0|
 5| 2.0| 0|
 9| 4.0| 0|
 11| 1.0| 0|
 12| 2.0| 0|
 15| 1.0| 0|
 17| 1.0| 0|
 19| 1.0| 0|
 21| 1.0| 0|
 23| 1.0| 0|
 26| 3.0| 0|
 27| 1.0| 0|
 28| 1.0| 0|
 29| 1.0| 0|
 30| 1.0| 0|
 31| 1.0| 0|
 34| 1.0| 0|
 37| 1.0| 0|
 41| 2.0| 0|
+-------+------+------+
only showing top 20 rows

In [6]:
df.describe().show()

+-------+------------------+------------------+------------------+
summary| movieId| rating| userId|
+-------+------------------+------------------+------------------+
 count| 1501| 1501| 1501|
 mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
 stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
 min| 0| 1.0| 0|
 max| 99| 5.0| 29|
+-------+------------------+------------------+------------------+

###Splitting the dataset

In [8]:
training_set, test_set = df.randomSplit([0.8, 0.2])

###Creating the model

In [10]:
from pyspark.ml.recommendation import ALS
recommender = ALS(userCol='userId', ratingCol='rating', itemCol='movieId')
recommender = recommender.fit(training_set)

###Predicting using the test set

In [12]:
preds = recommender.transform(test_set)

In [13]:
preds.show()

+-------+------+------+----------+
movieId|rating|userId|prediction|
+-------+------+------+----------+
 31| 1.0| 5| 1.8833612|
 31| 3.0| 8| 2.3561723|
 85| 1.0| 12| 1.5748006|
 85| 3.0| 1| 0.6428108|
 85| 5.0| 8| 2.1790695|
 85| 4.0| 7| 1.1771305|
 85| 3.0| 21| 1.4264474|
 65| 1.0| 28| 1.578831|
 65| 1.0| 22| 0.9933872|
 65| 2.0| 15| 1.4791101|
 53| 3.0| 13| 1.4423789|
 53| 1.0| 6| 1.7455294|
 53| 1.0| 25|0.24769796|
 53| 5.0| 21| 2.7853155|
 78| 1.0| 27| 0.8084004|
 34| 3.0| 3| 0.794008|
 34| 1.0| 19| 1.0743159|
 34| 1.0| 14| 1.3436971|
 81| 3.0| 26| 2.0795417|
 81| 1.0| 22|0.85718805|
+-------+------+------+----------+
only showing top 20 rows

###Evaluating the model

In [15]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol='rating')
evaluator.evaluate(preds)

Out[18]: 1.0920128352337675

###Making a recommendation

In [17]:
test_set.show()

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 0| 1.0| 23|
 0| 1.0| 26|
 1| 1.0| 6|
 1| 1.0| 19|
 1| 4.0| 15|
 2| 1.0| 16|
 2| 1.0| 25|
 2| 2.0| 1|
 2| 2.0| 7|
 2| 3.0| 0|
 2| 4.0| 10|
 2| 4.0| 28|
 3| 1.0| 1|
 3| 1.0| 9|
 3| 3.0| 14|
 4| 1.0| 7|
 4| 2.0| 1|
 4| 2.0| 20|
 4| 3.0| 2|
 4| 3.0| 10|
+-------+------+------+
only showing top 20 rows

In [18]:
test_set.filter(test_set['userId'] == 23).show()

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 0| 1.0| 23|
 26| 1.0| 23|
 32| 5.0| 23|
 38| 1.0| 23|
 59| 1.0| 23|
 66| 2.0| 23|
 67| 1.0| 23|
 82| 2.0| 23|
 97| 1.0| 23|
+-------+------+------+

In [19]:
single_user = test_set.filter(test_set['userId'] == 23).select(['userId', 'movieId'])

In [20]:
single_user.show()

+------+-------+
userId|movieId|
+------+-------+
 23| 0|
 23| 26|
 23| 32|
 23| 38|
 23| 59|
 23| 66|
 23| 67|
 23| 82|
 23| 97|
+------+-------+

In [21]:
recommendations = recommender.transform(single_user)

In [22]:
recommendations.orderBy('prediction', ascending=False).show()

+------+-------+----------+
userId|movieId|prediction|
+------+-------+----------+
 23| 32| 4.10642|
 23| 38| 2.863276|
 23| 66| 2.602256|
 23| 26| 2.5536926|
 23| 82| 2.327405|
 23| 97| 1.9184555|
 23| 0| 1.2491157|
 23| 59| 1.056267|
 23| 67| 0.7415104|
+------+-------+----------+